<h1 dir='rtl'>
تشخیص قصد و شناسایی عبارات زمانی، مکانی و... .
</h1>

<p dir='rtl'>
در ابتدا کتابخانه های مربوطه را نصب میکنیم
</p>

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!pip install -q transformers[torch]
!pip install -q --upgrade hazm
!pip install -q sentencepiece
!pip install -q torch
!pip install -q rich[jupyter]

<p dir='rtl'>
 حال از مدل هوشواره برای انجام تسک استفاده میکنیم.
 </p>

<p dir='rtl'>
  در ابتدا، مجموعه داده‌های "AmazonScience/massive" با استفاده از تابع "load_dataset" بارگیری می‌شود. سپس مجموعه داده به دو ستون جدید تغییر نام داده می‌شود: "utt" به "text" و "scenario" به "label".
 سپس تابع "clean_text" تعریف می‌شود که متن موجود در کلید "text" ورودی را با استفاده از تابع "normalize" شیء "normalizer" تغییر می‌دهد. در نهایت، مجموعه داده با استفاده از تابع "map" ورودی را به تابع "clean_text" اعمال می‌کند و نتیجه را در مجموعه داده جدید ذخیره می‌کند، در نهایت میحواهیم متن‌ها را نرمالیزه کنیم.

 </p>

<p dir='rtl'>
 تابع tokenize_function به وسیله تابع map بر روی مجموعه داده اعمال می‌شود، و نتیجه توکن‌بندی شده در مجموعه داده جدید با نام tokenized_datasets ذخیره می‌شود. توکن‌بندی بر روی داده‌ها به صورت دسته‌ای (batched) و با اندازه دسته‌های 128 توکن انجام می‌شود (batch_size=128).
 همچنین بیشترین طول یک رشته ورودی را ۶۴ میذاریم و اگر کابری متنی که یبشتر از ۶۴ باشد را اداعه کند، تا سر ۶۴ توکن را میپذیریم.
 </p>

<p dir='rtl'>
در اینجا، از تابع  np.argmax استفاده می‌شود تا اندیس عدد بزرگترین مقدار در هر سطر  logits  را برگرداند. به عبارت دیگر، این کد به ما اندیس پیش‌بینی‌ها را میدهد.

 </p>

<h3 dir='rtl'>

ابتدا، از کلاس  AutoModelForSequenceClassification  از ماژول  transformers  استفاده می‌شود تا یک مدل آماده برای دسته‌بندی توالی ایجاد شود. تابع  from_pretrained  بر روی این کلاس فراخوانی می‌شود تا مدل با استفاده از یک نام مدل یا مسیر آن ایجاد شود. همچنین تعداد برچسب‌های مختلف در دسته‌بندی را با استفاده از پارامتر  num_labels  مشخص می‌کنیم که در اینجا برابر با 7 است.

سپس، ویژگی  classifier  مدل را با یک لایه خطی جدید با استفاده از کلاس  nn.Linear  از ماژول  torch.nn جایگزین می‌کنیم. این لایه خطی ورودی با ابعاد 768 را به خروجی با ابعاد 18 تبدیل می‌کند. تنظیمات `bias=True` نشان می‌دهد که این لایه خطی یک بایاس دارد.

سپس، تعداد برچسب‌های مدل را به 18 تغییر می‌دهیم با تنظیم مقدار num_labels  بر روی 18. همچنین، دیکشنری  label2id  را به عنوان نگاشت از برچسب‌ها به شناسه‌ها و دیکشنری  id2label  را به عنوان نگاشت از شناسه‌ها به برچسب‌ها در تنظیمات مدل تغییر می‌دهیم. در اینجا، هر برچسب و شناسه متناظر با هم برابر است.

در نهایت، مدل نهایی برگشت داده می‌شود.
 </h3>

<p dir='rtl'>
ابتدا، یک شی  TrainingArguments  ایجاد می‌شود که پارامترهای آموزش را تعیین می‌کند. مقادیر پارامترهایی مانند  output_dir  (مسیر خروجی برای ذخیره مدل آموزش داده شده)،  evaluation_strategy  (استراتژی ارزیابی، در اینجا پس از هر دوره ارزیابی انجام می‌شود) و `num_train_epochs` (تعداد دوره‌های آموزش) تنظیم می‌شوند.
</p><p dir='rtl'>
سپس، یک شی  Trainer  ایجاد می‌شود که مسئول آموزش و ارزیابی مدل است. مدل آموزش داده شده ، پارامترهای آموزش (`args`)، مجموعه داده آموزش (`train_dataset`)، مجموعه داده ارزیابی و تابع compute_metrics (برای محاسبه معیارهای سنجش عملکرد) به عنوان ورودی به Trainer داده می‌شوند.
</p><p dir='rtl'>
در نتیجه، شی trainer حاوی مدل و پارامترهای آموزش است و می‌توان از آن برای آموزش و ارزیابی مدل استفاده کرد.
 </p>

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer ,AutoModelForSequenceClassification , TrainingArguments, Trainer, pipeline
from hazm import *
import numpy as np
import evaluate
from torch import nn


MODEL_NAME_OR_PATH = "HooshvareLab/bert-fa-base-uncased-clf-digimag"
dataset = load_dataset("AmazonScience/massive",'fa-IR')
dataset = dataset.rename_column("utt", "text")
dataset = dataset.rename_column("scenario", "label")
normalizer = Normalizer()

def clean_text(example):
    example["text"] = normalizer.normalize(example["text"])
    return example

dataset = dataset.map(clean_text)
print("the dataset:")
print(dataset)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=128)

metric = evaluate.load("accuracy")
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH, num_labels=7)
model.classifier = nn.Linear(in_features=768, out_features=18, bias=True)

model.num_labels = 18
model.config.label2id = {str(x): x for x in range(18)}
model.config.id2label = {str(x): str(x) for x in range(18)}

print("the model:")
print(model)

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

for param in model.bert.pooler.parameters():
    param.requires_grad = True

training_args = TrainingArguments(
    output_dir="dada_scenario_classifier",
    evaluation_strategy="epoch",
    num_train_epochs=10)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset massive downloaded and prepared to /root/.cache/huggingface/datasets/AmazonScience___massive/fa-IR/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

the dataset:
DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'label', 'intent', 'text', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'label', 'intent', 'text', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'label', 'intent', 'text', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})


Map:   0%|          | 0/11514 [00:00<?, ? examples/s]

Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

Map:   0%|          | 0/2974 [00:00<?, ? examples/s]

the model:
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [3]:
trainer.train()

pipe = pipeline(
    "text-classification", model=trainer.model, tokenizer=tokenizer, device=0
)

pipe.model.config.id2label = {
    int(k):v for k, v in pipe.model.config.id2label.items()}

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.660200,1.228682,0.650762
2,1.149800,0.947115,0.712248
3,0.974100,0.813908,0.761436
4,0.866400,0.746327,0.776193
5,0.804300,0.711730,0.789966
6,0.797900,0.679306,0.801279
7,0.758300,0.672735,0.796360
8,0.743700,0.658848,0.804230
9,0.750400,0.652909,0.805214
10,0.737400,0.647780,0.806198


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
labels = ['social',
 'transport',
 'calendar',
 'play',
 'news',
 'datetime',
 'recommendation',
 'email',
 'iot',
 'general',
 'audio',
 'lists',
 'qa',
 'cooking',
 'takeaway',
 'music',
 'alarm',
 'weather']

def predict(texts,correct_labels):
  cleaned_texts = [clean_text(txt) for txt in texts]
  predicted = pipe(cleaned_texts)
  for i,test in enumerate(predicted):
    print(texts[i]['text'])
    print('golden:',correct_labels[i],'\t','predicted:',labels[int(test['label'])])
    print('------')
  return predicted

<h3 dir='rtl'>
در نهایت هم تست میکنیم و خروجی میگیریم.
 </h3>

In [7]:
tests = [{"text": 'من را جمعه صبح ساعت نه بیدار کن'},  # 16 alarm
    {"text": 'صدای تلویزیون را تا حد امکان پایین بیار'}, #10 audio
    {"text": 'دیدن این صحنه و رفتار ناخوشایند است'}, #9 general
    {"text": 'باید پاستا را از رستوران تحویل گرفت و ببریم زیر پتو بخوریم'}, # 14 takeaway
    {"text":  'اولین روری که گمان کردیم که مال هم شدیم ۸ اردیبهشت بود'}, #2 calender
    {"text":  'چرا مریم رضوی بعر از ۲ سال شوور کردن به ما شیرینی نداد؟'}, #12 qa -----
    {"text": 'غذایت را ترکیبی نزن'}, #13 cooking
    {"text": 'آهنگ آخرین تلاش ستارو بزار حالم خوب نیست' }, # 15 music
    {"text": 'باید در کارهای خانه کمکم کنی مثلا با جاروبرقی تو اتاق ها را تمیز کنی' }, # 8 iot
    {"text": 'این روزها خیلی بد میگذرد و باید سریعا تمام شوند'}, # 5 datetime -----
    {"text": 'دوست دارم بریم زیر باران گریه کنیم' }, # 9 general
    {"text": 'برای تهیه پاستا باید خامه استفاده کنی' }, # 13 cooking
    {"text":  'باید چند هفته دیگر جلسه بگذاریم و منطقی صحبت کنیم' }, #2 calendar
    {"text":  'فکر میکردم کلی ایمیل اسپم ازت میگیرم اما در واقع هیچی ارسال نکرده بودی' }, # 7 email
    {"text": 'آهنگ the city holds my heart را پخش کن' }, # 3 play
    {"text": 'مردیم از این هوای گرم تابستان مسخره، پس کی برف میاد'}, # 17 weather
    {"text": ' تازگی وضعیت استوری و عکس های اینستاگرامم را نگاه نمیکنی'}, # 0 social -----
    {"text": 'به نظرت کتاب فکر آزاد بدون آدم ها جالب است؟' }, # 6  recomendation
    {"text": 'به من اطلاع بدید که هواپیمایی ماهان چه زمانی به دبی پرواز دارد؟' }, # 1 transport
    {"text": 'باید نگران حال هم باشیم پس برای مدتی، کمی فاصله نیاز است' }, #2 calendar
]
correct_labels = [
    'alarm',
    'iot',
    'general',
    'takeaway',
    'calender',
    'qa',
    'cooking' ,
    'play',
    'iot',
    'general',
    'general',
    'cooking' ,
'calendar',
    'email',
    'play',
    'weather',
    'social',
    'recomendation',
    'transport',
    'general'
]
predict(tests,correct_labels)

من را جمعه صبح ساعت نه بیدار کن
golden: alarm 	 predicted: alarm
------
صدای تلویزیون را تا حد امکان پایین بیار
golden: iot 	 predicted: audio
------
دیدن این صحنه و رفتار ناخوشایند است
golden: general 	 predicted: general
------
باید پاستا را از رستوران تحویل گرفت و ببریم زیر پتو بخوریم
golden: takeaway 	 predicted: takeaway
------
اولین روری که گمان کردیم که مال هم شدیم ۸ اردیبهشت بود
golden: calender 	 predicted: calendar
------
چرا مریم رضوی بعر از ۲ سال شوور کردن به ما شیرینی نداد؟
golden: qa 	 predicted: general
------
غذایت را ترکیبی نزن
golden: cooking 	 predicted: cooking
------
آهنگ آخرین تلاش ستارو بزار حالم خوب نیست
golden: play 	 predicted: music
------
باید در کارهای خانه کمکم کنی مثلا با جاروبرقی تو اتاق‌ها را تمیز کنی
golden: iot 	 predicted: iot
------
این روزها خیلی بد می‌گذرد و باید سریعا تمام شوند
golden: general 	 predicted: datetime
------
دوست دارم بریم زیر باران گریه کنیم
golden: general 	 predicted: general
------
برای تهیه پاستا باید خامه استفاده کنی
golden: c

[{'label': '16', 'score': 0.7245314717292786},
 {'label': '10', 'score': 0.6841864585876465},
 {'label': '9', 'score': 0.6341323852539062},
 {'label': '14', 'score': 0.5105888247489929},
 {'label': '2', 'score': 0.36554238200187683},
 {'label': '9', 'score': 0.21275372803211212},
 {'label': '13', 'score': 0.6026015877723694},
 {'label': '15', 'score': 0.478682279586792},
 {'label': '8', 'score': 0.7966445088386536},
 {'label': '5', 'score': 0.3270903527736664},
 {'label': '9', 'score': 0.318366140127182},
 {'label': '13', 'score': 0.8532017469406128},
 {'label': '2', 'score': 0.9825690984725952},
 {'label': '7', 'score': 0.6411353349685669},
 {'label': '3', 'score': 0.9321603775024414},
 {'label': '17', 'score': 0.8315722346305847},
 {'label': '7', 'score': 0.5265784859657288},
 {'label': '9', 'score': 0.4664296805858612},
 {'label': '1', 'score': 0.3442143499851227},
 {'label': '2', 'score': 0.5228391289710999}]